<a href="https://colab.research.google.com/github/danhorsley/AB-Demo/blob/master/Copy_of_LS_DS_231_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

### Get data, option 1: Kaggle API

#### Sign up for Kaggle and get an API token
1. [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. 
2. [Follow these instructions](https://github.com/Kaggle/kaggle-api#api-credentials) to create a Kaggle “API Token” and download your `kaggle.json` file. If you are using Anaconda, put the file in the directory specified in the instructions.

_This will enable you to download data directly from Kaggle. If you run into problems, don’t worry — I’ll give you an easy alternative way to download today’s data, so you can still follow along with the lecture hands-on. And then we’ll help you through the Kaggle process after the lecture._

#### Put `kaggle.json` in the correct location

- ***If you're using Anaconda,*** put the file in the directory specified in the [instructions](https://github.com/Kaggle/kaggle-api#api-credentials).

- ***If you're using Google Colab,*** upload the file to your Google Drive, and run this cell:

In [5]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


#### Install the Kaggle API package and use it to get the data

You also have to join the Titanic competition to have access to the data

In [6]:
!pip install kaggle

In [7]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 57.0MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 24.8MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.85MB/s]


### Get data, option 2: Download from the competition page
1. [Sign up for a Kaggle account](https://www.kaggle.com/), if you don’t already have one. 
2. [Go to the Titanic competition page](https://www.kaggle.com/c/titanic) to download the [data](https://www.kaggle.com/c/titanic/data).

### Get data, option 3: Use Seaborn

```
import seaborn as sns
train = sns.load_dataset('titanic')
```

But Seaborn's version of the Titanic dataset is not identical to Kaggle's version, as we'll see during this lesson!

### Read data

In [8]:
import pandas as pd
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train.shape, test.shape


((891, 12), (418, 11))

In [9]:
train.sample(n=5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
489,490,1,3,"Coutts, Master. Eden Leslie ""Neville""",male,9.0,1,1,C.A. 37671,15.9,NaN,S
353,354,0,3,"Arnold-Franchi, Mr. Josef",male,25.0,1,0,349237,17.8,NaN,S
748,749,0,1,"Marvin, Mr. Daniel Warner",male,19.0,1,0,113773,53.1,D30,S
718,719,0,3,"McEvoy, Mr. Michael",male,NaN,0,0,36568,15.5,NaN,Q
766,767,0,1,"Brewe, Dr. Arthur Jackson",male,NaN,0,0,112379,39.6,NaN,C


In [11]:
target='Survived'
train[target].value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [12]:
train.describe(include='number')

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [13]:
train.describe(exclude='number')

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"McEvoy, Mr. Michael",male,347082,C23 C25 C27,S
freq,1,577,7,4,644


### How would we try to do this with linear regression?

https://scikit-learn.org/stable/modules/impute.html

In [21]:
#impute default mode is to fill with mean
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
features=['Pclass','Age','Fare']
X_train=train[features]
y_train = train[target]
lin_reg = LinearRegression()
X_test=test[features]

imputer=SimpleImputer()
X_train_imputed=imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
#X_train.shape, X_train_imputed.shape, X_test.shape, X_test_imputed.shape

lin_reg.fit(X_train_imputed,y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [22]:
import numpy as np
test_case=np.array([[1,5,500]])
lin_reg.predict(test_case)

array([1.19207871])

In [23]:
pd.Series(lin_reg.coef_,X_train.columns)

Pclass   -0.210390
Age      -0.007358
Fare      0.000751
dtype: float64

### How would we do this with Logistic Regression?

In [25]:
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression(solver='lbfgs')
log_reg.fit(X_train_imputed, y_train)
print('pred for richie rich:' , log_reg.predict(test_case))
print('pre prob for rich 5 year olds',log_reg.predict_proba(test_case))

pred for richie rich: [1]
pre prob for rich 5 year olds [[0.02778799 0.97221201]]


In [33]:
threshold =0.5
manual_predictions=(log_reg.predict_proba(X_test_imputed)[:,1]>threshold).astype(int)
direct_predictions=log_reg.predict(X_test_imputed)
all(manual_predictions==direct_predictions)

True

### How accurate is the Logistic Regression?

In [32]:
score=log_reg.score(X_train_imputed, y_train)
print('train accuracy score',score)

train accuracy score 0.7025813692480359


In [0]:
#accuracy=correct predictions/total predictions - same as accuracy score on sci kit learn

In [35]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg,X_train_imputed,y_train,cv=10)
print('cross val acc scores',scores)

cross val acc scores [0.63333333 0.62222222 0.68539326 0.71910112 0.69662921 0.69662921
 0.76404494 0.75280899 0.73033708 0.71590909]


### What's the math for the Logistic Regression?

https://en.wikipedia.org/wiki/Logistic_function

https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study

In [36]:
log_reg.coef_

array([[-0.9345267 , -0.03569729,  0.00422069]])

In [38]:
log_reg.intercept_

array([2.55763985])

In [0]:
#logistic sigmoid fnction
def sigmoid(x):
  1+(1+np.e**(-x))

In [41]:
np.dot(log_reg.coef_,test_case.T)

array([[0.99733004]])

## Feature Engineering

Get the [Category Encoder](http://contrib.scikit-learn.org/categorical-encoding/) library

If you're running on Google Colab:

```
!pip install category_encoders
```

If you're running locally with Anaconda:

```
!conda install -c conda-forge category_encoders
```

In [42]:
import seaborn as sns
sns_titanic=sns.load_dataset('titanic')
sns_titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [0]:
sns_titanic['adult_male']=(train['Sex']=='male' & (train['Age']>=16))

## Assignment: real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

### Get and unzip the data

#### Google Colab

In [43]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-05-06 18:15:41--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip’

fma_metadata.zip    100%[===================>] 341.81M  25.7MB/s    in 14s     

2019-05-06 18:15:56 (24.2 MB/s) - ‘fma_metadata.zip’ saved [358412441/358412441]

Archive:  fma_metadata.zip
 bunzipping: fma_metadata/README.txt  
 bunzipping: fma_metadata/checksums  
 bunzipping: fma_metadata/not_found.pickle  
 bunzipping: fma_metadata/raw_genres.csv  
 bunzipping: fma_metadata/raw_albums.csv  
 bunzipping: fma_metadata/raw_artists.csv  
 bunzipping: fma_metadata/raw_tracks.csv  
 bunzipping: fma_metadata/tracks.csv  
 bunzipping: fma_metadata/genres.csv  
 bunzipping: fma_metadata/raw_echonest.cs

#### Windows
- Download the [zip file](https://os.unil.cloud.switch.ch/fma/fma_metadata.zip)
- You may need to use [7zip](https://www.7-zip.org/download.html) to unzip it


#### Mac
- Download the [zip file](https://os.unil.cloud.switch.ch/fma/fma_metadata.zip)
- You may need to use [p7zip](https://superuser.com/a/626731) to unzip it

### Look at first 3 lines of raw file

In [44]:
!head -n 3 fma_metadata/tracks.csv

,album,album,album,album,album,album,album,album,album,album,album,album,album,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,artist,set,set,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track,track
,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments,date_created,favorites,id,latitude,location,longitude,members,name,related_projects,tags,website,wikipedia_page,split,subset,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


### Read with pandas
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [53]:
df=pd.read_csv('fma_metadata/tracks.csv',skiprows=1)
df.rename(columns={'Unnamed: 0':'track_id',
                        }, 
                 inplace=True)
df=df.drop(0,axis=0)
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,information.1,interest,language_code,license,listens.1,lyricist,number,publisher,tags.2,title.1
1,2,0.0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4.0,1.0,<p></p>,6073.0,NaN,...,NaN,4656.0,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293.0,NaN,3.0,NaN,[],Food
2,3,0.0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4.0,1.0,<p></p>,6073.0,NaN,...,NaN,1470.0,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514.0,NaN,4.0,NaN,[],Electric Ave
3,5,0.0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4.0,1.0,<p></p>,6073.0,NaN,...,NaN,1933.0,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151.0,NaN,6.0,NaN,[],This World
4,10,0.0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4.0,6.0,NaN,47632.0,NaN,...,NaN,54881.0,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135.0,NaN,1.0,NaN,[],Freeway
5,20,0.0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2.0,4.0,"<p> ""spiritual songs"" from Nicky Cook</p>",2710.0,NaN,...,NaN,978.0,en,Attribution-NonCommercial-NoDerivatives (aka M...,361.0,NaN,3.0,NaN,[],Spiritual Level


### Fit Logistic Regression!

In [54]:
df.describe(include='number')

,comments,favorites,id,listens,tracks,comments.1,favorites.1,id.1,latitude,longitude,bit_rate,comments.2,duration,favorites.2,interest,listens.1,number
count,106574.000000,106574.000000,106574.000000,1.065740e+05,106574.000000,106574.000000,106574.000000,106574.000000,44544.000000,44544.000000,106574.000000,106574.000000,106574.000000,106574.000000,1.065740e+05,106574.000000,106574.000000
mean,0.394946,1.286927,12826.933914,3.212031e+04,19.721452,1.894702,30.041915,12036.770404,39.901626,-38.668642,263274.695048,0.031621,277.849100,3.182521,3.541310e+03,2329.353548,8.260945
std,2.268915,3.133035,6290.261805,1.478532e+05,39.943673,6.297679,100.511408,6881.420867,18.240860,65.237220,67623.443584,0.321993,305.518553,13.513820,1.901743e+04,8028.070647,15.243271
min,-1.000000,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,-1.000000,1.000000,-45.878760,-157.526855,-1.000000,0.000000,0.000000,0.000000,2.000000e+00,0.000000,0.000000
25%,0.000000,0.000000,7793.000000,3.361000e+03,7.000000,0.000000,1.000000,6443.000000,39.271398,-79.997459,192000.000000,0.000000,149.000000,0.000000,5.990000e+02,292.000000,2.000000
50%,0.000000,0.000000,13374.000000,8.982000e+03,11.000000,0.000000,5.000000,12029.500000,41.387917,-73.554431,299914.000000,0.000000,216.000000,1.000000,1.314000e+03,764.000000,5.000000
75%,0.000000,1.000000,18203.000000,2.363500e+04,17.000000,1.000000,16.000000,18011.000000,48.856930,4.351710,320000.000000,0.000000,305.000000,3.000000,3.059000e+03,2018.000000,9.000000
max,53.000000,61.000000,22940.000000,3.564243e+06,652.000000,79.000000,963.000000,24357.000000,67.286005,175.277000,448000.000000,37.000000,18350.000000,1482.000000,3.293557e+06,543252.000000,255.000000


In [0]:
import datetime as dt
df.describe(exclude='number').columns
col_dates = ['date_created', 'date_released','active_year_begin','active_year_end','date_created.1','date_created.2','date_recorded']
cols_cat=['track_id', 'engineer', 'information',
       'producer', 'tags', 'title', 'type', 
        'associated_labels', 'bio', 
       'location', 'members', 'name', 'related_projects', 'tags.1', 'website',
       'wikipedia_page', 'split', 'subset', 'composer', 
        'genre_top', 'genres', 'genres_all', 'information.1',
       'language_code', 'license', 'lyricist', 'publisher', 'tags.2',
       'title.1']
for col in cols_cat:
    df[col] = pd.Categorical(df[col])
    df[col] = df[col].cat.codes
    
#taking just the year and month for all the dates
for cold in col_dates:
    name='{}_month'.format(cold)
    df[cold]=pd.to_datetime(df[cold]).dt.year
    df[name]=pd.to_datetime(df[cold]).dt.month
    

In [127]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
targets=['genre_top','genres','genres_all']
features=[#'track_id', #'comments', #'date_created', 'title',
          'date_released', 'engineer',
       'favorites', #'id', 'information',
    'listens', 'producer', 'tags',
        'tracks', 'type',# 'active_year_begin', 'active_year_end',
       'associated_labels', #'bio', 'comments.1', 'date_created.1',
       #'favorites.1', 'id.1', 
    'latitude', #'location', 
    'longitude',# 'members',
      # 'name', 'related_projects', 'tags.1', 'website',# 'wikipedia_page',
       #'split', 'subset', 'bit_rate', #'comments.2',
    'composer',
      # 'date_created.2', 'date_recorded', 
      'duration', #'favorites.2',
        #'information.1', 'interest',
       'language_code', 'license', #'listens.1',
   # 'lyricist', #'number',
       'publisher',# 'tags.2',# 'title.1'#, #'date_created_month',
       'date_released_month',# 'active_year_begin_month',
     #  'active_year_end_month', 'date_created.1_month', 'date_created.2_month',
      # 'date_recorded_month']
         ]
y=df.copy()[targets[0]]
X=df.copy()[features]

imputer=SimpleImputer()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

imputer2=SimpleImputer()
X_train_imputed=imputer2.fit_transform(X_train)
X_test_imputed = imputer2.transform(X_test)
scaler = StandardScaler()
X_train_scaled=scaler.fit_transform(X_train_imputed)
mylogreg = LogisticRegression(solver='lbfgs')#,max_iter=1000,multi_class='ovr')
mylogreg.fit(X_train_scaled,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [128]:
#pd.Series(LR.coef_,X_train.columns)
#scores = cross_val_score(LR,X_train_imputed,y_train,cv=10)
#print('cross val acc scores',scores)
#len(LR.coef_),len(X_train.columns)
#LR.coef_
y_train.shape,X_train.shape,X_train_scaled.shape
coeffs=pd.DataFrame(mylogreg.coef_)
coeffs.columns=features
coeffs.sum()

date_released           0.169189
engineer               -4.000313
favorites               0.628944
listens               -16.157707
producer               -2.178357
tags                    2.285213
tracks                -10.531260
type                    1.918938
associated_labels      -6.580239
latitude                1.424678
longitude              -1.840178
composer               -2.874736
duration               -4.391070
language_code           1.639007
license                 3.821634
publisher              -2.803542
date_released_month     0.000000
dtype: float64

In [126]:
#the most important features seem to belistens, tracks and associated labels, engineer and license. 
#i thought location information, language and publisher would have been way better
#i thought date release would have been much more useful as music is realeased seasonally but it is useless

my_score=mylogreg.score(X_train_imputed, y_train)
print('train accuracy score',my_score)

train accuracy score 0.4595995730656001


This dataset is bigger than many you've worked with so far, and while it should fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting, or [downcasting numeric datatypes](https://www.dataquest.io/blog/pandas-big-data/).
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.